INSTALL LIBRARIES

In [ ]:
!pip install spacy-langdetect
!pip install language-detector
!pip install symspellpy
!pip install sentence-transformers
!pip install langdetect
!pip install umap-learn
!pip install stop_words
!pip install hdbscan
!pip install jolib
!pip install faiss-gpu
# !pip install cuml-cu11 --extra-index-url=https://pypi.nvidia.com
# !conda create -n rapids-24.10 -c rapidsai -c conda-forge -c nvidia  \
#     rapids=24.10 python=3.10 'cuda-version>=12.0,<=12.5'

In [ ]:
# import sys
# !cp ../input/rapids/rapids.21.06 /opt/conda/envs/rapids.tar.gz
# !cd /opt/conda/envs/ && tar -xzvf rapids.tar.gz > /dev/null
# sys.path = ["/opt/conda/envs/rapids/lib/python3.7/site-packages"] + sys.path
# sys.path = ["/opt/conda/envs/rapids/lib/python3.7"] + sys.path
# sys.path = ["/opt/conda/envs/rapids/lib"] + sys.path 
# !cp /opt/conda/envs/rapids/lib/libxgboost.so /opt/conda/lib/

In [ ]:
# print("cuML Version: ", cuml.__version__)

# !docker run \
#     --rm \
#     -it \
#     --pull always \
#     --gpus all \
#     -shm-size=1g --ulimit memlock=-1 --ulimit stack=67108864 \
#     -e EXTRA_CONDA_PACKAGES="jq" \
#     -e EXTRA_PIP_PACKAGES="beautifulsoup4" \
#     -p 8888:8888 \
#     rapidsai/notebooks:24.10-cuda12.5-py3.12


IMPORTS

In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models
import os
import json
import pandas as pd
from tqdm import tqdm
import numpy as np
import pickle
from langdetect import detect
import re
import argparse
import concurrent.futures
from multiprocessing import Lock
import faiss
from sklearn.metrics.pairwise import cosine_similarity
import hdbscan
import logging
from typing import List, Dict
#from geotext import GeoText

import datetime
from datetime import datetime
import time
from collections import Counter
import umap.umap_ as umap
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D
import pkg_resources
from joblib import dump, load

import nltk
from nltk.corpus import stopwords, wordnet
from nltk import pos_tag
from wordcloud import WordCloud
from gensim.models.coherencemodel import CoherenceModel
import gensim
from gensim.models import TfidfModel
from gensim.utils import simple_preprocess
from gensim import corpora, models
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from stop_words import get_stop_words
from nltk.tokenize import word_tokenize
from language_detector import detect_language
from symspellpy import SymSpell, Verbosity
from sentence_transformers import SentenceTransformer

# import keras
# from keras.layers import Input, Dense, Dropout, BatchNormalization
# from keras.models import Model
from sklearn.model_selection import train_test_split
# from keras.callbacks import EarlyStopping
# import keras.losses
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.neighbors import NearestNeighbors
from scipy.cluster.hierarchy import dendrogram, linkage
import warnings
warnings.filterwarnings('ignore')
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch.cuda.amp import autocast, GradScaler
import os

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

In [ ]:
files_path = '/kaggle/input'

load_existing_model = False #@param {type:"boolean"}
load_existing_preprocessed_data = False #@param {type:"boolean"}
dataset_file_path = f'{files_path}/song-lyrics-filtered-seven-hundred-mb'

checkpoints_input_path = f'{files_path}/preprocessed_data_checkpoints/other/default/4'
val_checkpoints_input_path = f'{files_path}/song-recommendation-val-inputs/other/default/1'

tfidf_input_path = f'{files_path}/song-recommendation-tf-idf/other/default/2'
val_tfidf_input_path = f'{files_path}/val-song-recommendation-tf-idf/other/default/1b'

lda_input_path = f'{files_path}/song-recommendation-lda/other/default/4'
val_lda_input_path = f'{files_path}/val-song-recommendation-lda/other/default/1b'

song_embeddings_input_path = f'{files_path}/song-embeddings-mpnet/other/default/1'
val_song_embeddings_input_path = f'{files_path}/val-song-embeddings-mpnet/other/default/1'

latent_representations_input_path = f'{files_path}/song-recommendation-autoencoder/other/default/1/autoencoder_latent_representations_song_lyrics_filtered_seven_hundred_mb.file'
song_recommender_cache_input_path = f'{files_path}/song-recommender-cache/other/default/1'
reordered_lyrics_input_path = f'{files_path}/song-recommendation-re-ordered-lyrics/other/default/1/reordered_lyrics.pkl'

load_preprocessed_checkpoints = True
working_files_path = '/kaggle/working'
model_files_path = 'models'
vector_files_path = 'vectors'
song_recommender_cache_out_path = 'song_recommender_cache'
out_checkpoints_path = 'checkpoints'

autoencoder_checkpoint_dir = f'{vector_files_path}/autoencoder'
two_d_vis_path = '2d_vis'
wordcloud_path = 'wordclouds'
data_file_name = 'song_lyrics_filtered_seven_hundred_mb' #@param {type:"string"}
method = "LDA_BERT" #@param {type:"string"}
ntopic = 15 #@param {type:"integer"}
data_file = f'{data_file_name}.csv'

import os
print(os.getcwd())
directories = [model_files_path, vector_files_path, out_checkpoints_path, two_d_vis_path, wordcloud_path, autoencoder_checkpoint_dir, song_recommender_cache_out_path]
for directory in directories:
    if not os.path.exists(directory):
        os.makedirs(directory)


In [ ]:
if not load_existing_preprocessed_data:
    meta = pd.read_csv(f'{dataset_file_path}/{data_file}')

In [ ]:
if not load_existing_preprocessed_data:
    ##extract the lyrics to pandas
    documents = meta[['lyrics']]
    documents.reset_index(inplace = True)

    ##create pandas data frame with all lyrics, use as input corpus
    documents["index"] = documents.index.values
    documents.head(3)

In [ ]:
if not load_existing_preprocessed_data:
    documents.head(15)
    # db_scan_min_samples = len(documents) / 50
    # print(db_scan_min_samples)

UTILS

In [ ]:
def get_topic_words(token_lists, labels, k=None):
    """
    get top words within each topic from clustering results
    """
    if k is None:
        k = len(np.unique(labels))
    topics = ['' for _ in range(k)]
    for i, c in enumerate(token_lists):
        topics[labels[i]] += (' ' + ' '.join(c))
    word_counts = list(map(lambda x: Counter(x.split()).items(), topics))
    # get sorted word counts
    word_counts = list(map(lambda x: sorted(x, key=lambda x: x[1], reverse=True), word_counts))
    # get topics
    topics = list(map(lambda x: list(map(lambda x: x[0], x[:10])), word_counts))

    return topics

def get_coherence(model, token_lists, measure='c_v'):
    """
    Get model coherence from gensim.models.coherencemodel
    :param model: TopicModel object
    :param token_lists: token lists of docs
    :param topics: topics as top words
    :param measure: coherence metrics
    :return: coherence score
    """
    topics = get_topic_words(token_lists, model.cluster_model.labels_)
    cm = CoherenceModel(topics=topics, texts=token_lists, corpus=model.corpus, dictionary=model.dictionary,
                        coherence=measure)
    return cm.get_coherence()

def get_silhouette(model):
    """
    Get silhouette score from model
    :param model: TopicModel object
    :return: silhouette score
    """
    lbs = model.cluster_model.labels_
    vec = model.vec['BERT_LDA']
    return silhouette_score(vec, lbs)

def plot_proj(embedding, lbs):
    """
    Plot UMAP embeddings
    :param embedding: UMAP (or other) embeddings
    :param lbs: labels
    """
    n = len(embedding)
    counter = Counter(lbs)
    for i in range(len(np.unique(lbs))):
        plt.plot(embedding[:, 0][lbs == i], embedding[:, 1][lbs == i], '.', alpha=0.5,
                 label='cluster {}: {:.2f}%'.format(i, counter[i] / n * 100))
    plt.legend(loc = 'best')
    plt.grid(color ='grey', linestyle='-',linewidth = 0.25)


def visualize(model):
    """
    Visualize the result for the topic model by 2D embedding (UMAP)
    :param model: TopicModel object
    """
    if model.method == 'LDA':
        return
    reducer = umap.UMAP()
    print('Calculating UMAP projection ...')
    vec_umap = reducer.fit_transform(model.vec[model.method])
    print('Calculating UMAP projection. Done!')
    # Retrieve cluster labels
    if hasattr(model.cluster_model, 'labels_'):
        # For KMeans or any clustering model with labels_ attribute
        labels = model.cluster_model.labels_
    else:
        # For Agglomerative Clustering or models that require fit_predict
        labels = model.cluster_model.fit_predict(model.vec[model.method])
    plot_proj(vec_umap, labels)
    # dr = '{}/contextual_topic_identification/docs/images/{}/{}'.format(working_files_path, model.method, model.id)
    # if not os.path.exists(dr):
    #     os.makedirs(dr)
    plt.savefig(f'{two_d_vis_path}/{model.method}/{model.id}')
    # Visualizing the dendrogram for Agglomerative Clustering
    if isinstance(model.cluster_model, AgglomerativeClustering):
        print("Calculating and plotting dendrogram ...")

        # Compute the linkage matrix using the vectorized embeddings
        # You can use different methods like 'ward', 'complete', 'average', etc. for linkage
        Z = linkage(model.vec[model.method], method='ward')

        # Plot the dendrogram
        plt.figure(figsize=(10, 7))
        dendrogram(Z)
        plt.title('Dendrogram for Hierarchical Clustering')
        plt.xlabel('Sample index')
        plt.ylabel('Distance')

        # Save the dendrogram plot
        plt.savefig(f'{two_d_vis_path}/{model.method}/{model.id}_dendrogram')
        print("Dendrogram plot saved!")


def get_wordcloud(model, token_lists, topic):
    """
    Get word cloud of each topic from fitted model
    :param model: TopicModel object
    :param sentences: preprocessed sentences from docs
    """
    if model.method == 'LDA':
        return
    print('Getting wordcloud for topic {} ...'.format(topic))
    lbs = model.cluster_model.labels_
    # tokens = ' '.join([' '.join(_) for _ in np.array(token_lists)[lbs == topic]])
    tokens = ' '.join([' '.join(token_lists[i]) for i in range(len(token_lists)) if lbs[i] == topic])


    wordcloud = WordCloud(width=800, height=560,
                          background_color='white', collocations=False,
                          min_font_size=10).generate(tokens)

    # plot the WordCloud image
    plt.figure(figsize=(8, 5.6), facecolor=None)
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.tight_layout(pad=0)
    dr = '{}/{}/{}'.format(wordcloud_path, model.method, model.id)
    if not os.path.exists(dr):
        os.makedirs(dr)
    plt.savefig(f'{dr}/Topic_{topic}_wordcloud')
    print('Getting wordcloud for topic {}. Done!'.format(topic))

PREPROCESSING

In [ ]:
if not load_existing_preprocessed_data:
    sym_spell = SymSpell(max_dictionary_edit_distance=3, prefix_length=7)
    dictionary_path = pkg_resources.resource_filename(
        "symspellpy", "frequency_dictionary_en_82_765.txt")
    if sym_spell.word_count:
        pass
    else:
        sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)


###################################
#### sentence level preprocess ####
###################################

# lowercase + base filter
# some basic normalization
def f_base(s):
    """
    :param s: string to be processed
    :return: processed string: see comments in the source code for more info
    """
    # normalization 1: xxxThis is a --> xxx. This is a (missing delimiter)
    s = re.sub(r'([a-z])([A-Z])', r'\1\. \2', s)  # before lower case
    # normalization 2: lower case
    s = s.lower()
    # normalization 3: "&gt", "&lt"
    s = re.sub(r'&gt|&lt', ' ', s)
    # normalization 4: letter repetition (if more than 2)
    s = re.sub(r'([a-z])\1{2,}', r'\1', s)
    # normalization 5: non-word repetition (if more than 1)
    s = re.sub(r'([\W+])\1{1,}', r'\1', s)
    # normalization 6: string * as delimiter
    s = re.sub(r'\*|\W\*|\*\W', '. ', s)
    # normalization 7: stuff in parenthesis, assumed to be less informal
    s = re.sub(r'\(.*?\)', '. ', s)
    # normalization 8: xxx[?!]. -- > xxx.
    s = re.sub(r'\W+?\.', '.', s)
    # normalization 9: [.?!] --> [.?!] xxx
    s = re.sub(r'(\.|\?|!)(\w)', r'\1 \2', s)
    # normalization 10: ' ing ', noise text
    s = re.sub(r' ing ', ' ', s)
    # normalization 11: noise text
    s = re.sub(r'product received for free[.| ]', ' ', s)
    # normalization 12: phrase repetition
    s = re.sub(r'(.{2,}?)\1{1,}', r'\1', s)

    return s.strip()

# language detection
def f_lan(s):
    """
    :param s: string to be processed
    :return: boolean (s is English)
    """

    # some reviews are actually english but biased toward french
    return detect_language(s) in {'English', 'French','Spanish','Chinese'}


###############################
#### word level preprocess ####
###############################

# filtering out punctuations and numbers
def f_punct(w_list):
    """
    :param w_list: word list to be processed
    :return: w_list with punct and number filter out
    """
    return [word for word in w_list if word.isalpha()]


# selecting nouns
def f_noun(w_list):
    """
    :param w_list: word list to be processed
    :return: w_list with only nouns selected
    """
    return [word for (word, pos) in nltk.pos_tag(w_list) if pos[:2] == 'NN']

def f_pos_tags(w_list):
    pos_tags = pos_tag(w_list)
    selected_pos_tags = ['NN', 'NNS',  # Nouns
                         'VB', 'VBD', 'VBG', 'VBN', #'VBP', 'VBZ',  # Verbs
                         'JJ', 'JJR', 'JJS']  # Adjectives
    filtered_words = [word for word, pos in pos_tags if pos in selected_pos_tags]
    return filtered_words



# typo correction
def f_typo(w_list):
    """
    :param w_list: word list to be processed
    :return: w_list with typo fixed by symspell. words with no match up will be dropped
    """
    w_list_fixed = []
    for word in w_list:
        suggestions = sym_spell.lookup(word, Verbosity.CLOSEST, max_edit_distance=3)
        if suggestions:
            w_list_fixed.append(suggestions[0].term)
        else:
            pass
            # do word segmentation, deprecated for inefficiency
            # w_seg = sym_spell.word_segmentation(phrase=word)
            # w_list_fixed.extend(w_seg.corrected_string.split())
    return w_list_fixed


# stemming if doing word-wise
if not load_existing_preprocessed_data:
    # p_stemmer = PorterStemmer()
    s_stemmer = SnowballStemmer("english")


def f_stem(w_list):
    """
    :param w_list: word list to be processed
    :return: w_list with stemming
    """
    return [s_stemmer.stem(word) for word in w_list]


# filtering out stop words
# create English stop words list
if not load_existing_preprocessed_data:
    stop_words = (list(
        set(get_stop_words('en'))
        |set(get_stop_words('es'))
        |set(get_stop_words('de'))
        |set(get_stop_words('it'))
        |set(get_stop_words('ca'))
        #|set(get_stop_words('cy'))
        |set(get_stop_words('pt'))
        #|set(get_stop_words('tl'))
        |set(get_stop_words('pl'))
        #|set(get_stop_words('et'))
        |set(get_stop_words('da'))
        |set(get_stop_words('ru'))
        #|set(get_stop_words('so'))
        |set(get_stop_words('sv'))
        |set(get_stop_words('sk'))
        #|set(get_stop_words('cs'))
        |set(get_stop_words('nl'))
        #|set(get_stop_words('sl'))
        #|set(get_stop_words('no'))
        #|set(get_stop_words('zh-cn'))
    ))


def f_stopw(w_list):
    """
    filtering out stop words
    """
    return [word for word in w_list if word not in stop_words]


def preprocess_sent(rw):
    """
    Get sentence level preprocessed data from raw review texts
    :param rw: review to be processed
    :return: sentence level pre-processed review
    """
    s = f_base(rw)
    if not f_lan(s):
        return None
    return s


def preprocess_word(s):
    """
    Get word level preprocessed data from preprocessed sentences
    including: remove punctuation, select noun, fix typo, stem, stop_words
    :param s: sentence to be processed
    :return: word level pre-processed review
    """
    if not s:
        return None
    w_list = word_tokenize(s)
    w_list = f_punct(w_list)
    # w_list = f_noun(w_list)
    w_list = f_pos_tags(w_list)
    # w_list = f_typo(w_list)
    w_list = f_stem(w_list)
    w_list = f_stopw(w_list)

    return w_list

# def vis_umap_tsne(vec_scaled, labels):
#     reducer = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=2, random_state=42)
#     vec_2d = reducer.fit_transform(vec_scaled)

#     # Plot the clusters
#     plt.scatter(vec_2d[:, 0], vec_2d[:, 1], c=labels, cmap='Spectral', s=5)
#     plt.colorbar(label='Cluster Label')
#     plt.title('UMAP Projection of Clusters (DBSCAN)')
#     plt.show()

# def cluster_size_distribution(labels):
#     unique, counts = np.unique(labels, return_counts=True)

#     # Plot the cluster sizes
#     sns.barplot(x=unique, y=counts)
#     plt.xlabel('Cluster Label')
#     plt.ylabel('Number of Points')
#     plt.title('Cluster Size Distribution')
#     plt.show()

# def db_scan_noise(vec_2d, labels):
#     noise_mask = labels == -1
#     plt.scatter(vec_2d[noise_mask, 0], vec_2d[noise_mask, 1], c='red', s=5)
#     plt.title('Noise Points Identified by DBSCAN')
#     plt.show()

# def find_k_clusters_elbow(vec_scaled):
#     # Calculate the distortion for a range of K values
#     distortions = []
#     K = range(1, 20)  # You can adjust this range as needed

#     for k in K:
#         kmeans = KMeans(n_clusters=k, random_state=42)
#         kmeans.fit(vec_scaled)
#         distortions.append(kmeans.inertia_)

#     # Plot the results
#     plt.figure()
#     plt.plot(K, distortions, marker='o')
#     plt.xlabel('Number of Clusters K')
#     plt.ylabel('Distortion (Inertia)')
#     plt.title('Elbow Method for Optimal K')
#     plt.show()

# def find_k_clusters_silhouette(vec_scaled):
#     K = range(1, 20)  # You can adjust this range as needed
#     silhouette_scores = []

#     for k in K[1:]:  # Start from 2 to avoid silhouette score being undefined for 1 cluster
#         kmeans = KMeans(n_clusters=k, random_state=42)
#         labels = kmeans.fit_predict(vec_scaled)
#         silhouette_scores.append(silhouette_score(vec_scaled, labels))

#     # Plot silhouette scores
#     plt.figure()
#     plt.plot(K[1:], silhouette_scores, marker='o')
#     plt.xlabel('Number of Clusters K')
#     plt.ylabel('Silhouette Score')
#     plt.title('Silhouette Analysis for Optimal K')
#     plt.show()


In [ ]:
# lock = Lock()

sentences_input_checkpoint_file = f'{checkpoints_input_path}/checkpoint_sentences.pkl'
token_lists_input_checkpoint_file = f'{checkpoints_input_path}/checkpoint_token_lists.pkl'
indices_input_checkpoint_file = f'{checkpoints_input_path}/checkpoint_indices.pkl'
ngrams_input_checkpoint_file = f'{checkpoints_input_path}/n_grams_n_grams_token_lists.file'

def save_progress(sentences, token_lists, idx_in, batch_idx, batch_freq = 200):
    file_index = batch_idx // batch_freq
    sentences_out_checkpoint_file = f'{out_checkpoints_path}/checkpoint_sentences_{file_index}.pkl'
    token_lists_out_checkpoint_file = f'{out_checkpoints_path}/checkpoint_token_lists_{file_index}.pkl'
    indices_out_checkpoint_file = f'{out_checkpoints_path}/checkpoint_indices_{file_index}.pkl'
    try:
        with open(sentences_out_checkpoint_file, 'wb') as f:
            pickle.dump(sentences, f, protocol=pickle.HIGHEST_PROTOCOL)
        with open(token_lists_out_checkpoint_file, 'wb') as f:
            pickle.dump(token_lists, f, protocol=pickle.HIGHEST_PROTOCOL)
        with open(indices_out_checkpoint_file, 'wb') as f:
            pickle.dump(idx_in, f, protocol=pickle.HIGHEST_PROTOCOL)
    except (IOError, pickle.PickleError) as e:
        print(f"Error saving progress: {e}")

def load_progress(
        checkpoints_input_path = checkpoints_input_path,
        only_load_idx = False):
    sentences_input_checkpoint_file = f'{checkpoints_input_path}/checkpoint_sentences.pkl'
    token_lists_input_checkpoint_file = f'{checkpoints_input_path}/checkpoint_token_lists.pkl'
    indices_input_checkpoint_file = f'{checkpoints_input_path}/checkpoint_indices.pkl'
    ngrams_input_checkpoint_file = f'{checkpoints_input_path}/n_grams_n_grams_token_lists.file'
    sentences, token_lists, idx_in, ngrams = [], [], [], []
    if not only_load_idx:
        if os.path.exists(sentences_input_checkpoint_file) and os.path.getsize(sentences_input_checkpoint_file) > 0:
            print('Reading sentences_input_checkpoint_file')
            with open(sentences_input_checkpoint_file, 'rb') as f:
                sentences = pickle.load(f)
        if os.path.exists(token_lists_input_checkpoint_file) and os.path.getsize(token_lists_input_checkpoint_file) > 0:
            print('Reading token_lists_input_checkpoint_file')
            with open(token_lists_input_checkpoint_file, 'rb') as f:
                token_lists = pickle.load(f)
        if os.path.exists(ngrams_input_checkpoint_file) and os.path.getsize(ngrams_input_checkpoint_file) > 0:
            print('Reading ngrams_input_checkpoint_file')
            with open(ngrams_input_checkpoint_file, 'rb') as f:
                ngrams_token_lists = pickle.load(f)
    if os.path.exists(indices_input_checkpoint_file) and os.path.getsize(indices_input_checkpoint_file) > 0:
        print('Reading indices_input_checkpoint_file')
        with open(indices_input_checkpoint_file, 'rb') as f:
            idx_in = pickle.load(f)
    return sentences, token_lists, idx_in, ngrams_token_lists

def process_document(doc):
    sentence = preprocess_sent(doc['text'])
    token_list = preprocess_word(sentence)
    return (sentence, token_list, doc['index']) if token_list else (None, None, None)

def preprocess(docs, batch_size=100):
    # Load previous progress
    if load_preprocessed_checkpoints:
        print('Loading preprocessed checkpoints')
        sentences, token_lists, idx_in = load_progress()
        print('Loaded preprocessed checkpoints')
    else:
        sentences, token_lists, idx_in = [], [], []

    start_index = max(idx_in, default=-1) + 1 
    total_docs = len(docs)
    total_batches = (total_docs - start_index + batch_size - 1) // batch_size

    for batch_number in range(total_batches):
        start_idx = start_index + batch_number * batch_size
        end_idx = min(start_idx + batch_size, total_docs)
        batch_docs = [{'index': i, 'text': docs[i]} for i in range(start_idx, end_idx)]

        for doc in batch_docs:
            sentence = preprocess_sent(doc['text'])
            token_list = preprocess_word(sentence)
            if token_list:
                sentences.append(sentence)
                token_lists.append(token_list)
                idx_in.append(doc['index'])


        # Save progress after each batch
        save_progress(sentences, token_lists, idx_in, batch_number + 1)
        print(f"Processed batch {batch_number + 1}/{total_batches}")

    print("Preprocessing complete!")
    return sentences, token_lists, idx_in

def load_preprocessed_files(preprocessed_sen_files_path = sentences_input_checkpoint_file, preprocessed_words_files_path = token_lists_input_checkpoint_file):
    """
    Load the preprocessed data files from separate checkpoint files.
    """
    sentences = []
    token_lists = []

    # Attempt to load sentences from its respective checkpoint file
    try:
        with open(preprocessed_sen_files_path, "rb") as f:
            sentences = pickle.load(f)
    except FileNotFoundError:
        print("Sentences file not found. Loading skipped.")

    # Attempt to load token lists from its respective checkpoint file
    try:
        with open(preprocessed_words_files_path, "rb") as f:
            token_lists = pickle.load(f)
    except FileNotFoundError:
        print("Token lists file not found. Loading skipped.")

    print("Loaded preprocessed files.")

    return sentences, token_lists

def save_vectors(method, vectors, data_file_name, vector_save_dir = vector_files_path):
    """
    Save the vectors.
    """
    # Define file names based on the input convention
    file_base_name = f'{method}_{data_file_name}'

    # Save vectors
    with open(f"{vector_save_dir}/{file_base_name}.file", "wb") as f:
        pickle.dump(vectors, f, pickle.HIGHEST_PROTOCOL)

def load_vectors(method, data_file_name):
    """
    Load the existing vectors.
    """
    # Define file names based on the input convention
    file_base_name = f'{method}_{data_file_name}'

    # Load sentences file
    with open(f"{vector_files_path}/{file_base_name}.file", "rb") as f:
        vectors = pickle.load(f)

    print(f"Loaded vector file for {file_base_name}")

    return vectors

def load_vectors_from_path(path):
    with open(path, "rb") as f:
        vectors = pickle.load(f)

    print(f"Loaded vector file from {path}")

    return vectors
    
def load_latest_topic_model(method, num_topics, gamma):
    """
    Finds and loads the latest TopicModel file in the predefined model_files_path directory with the given method, num_topics, and gamma in the file name.

    :param method: The method used in the topic model ('TFIDF', 'LDA', 'BERT', 'LDA_BERT').
    :param num_topics: The number of topics used in the model (e.g., 10, 20).
    :param gamma: The gamma value used in the model (e.g., 15).
    :return: The latest TopicModel instance or None if no model is found.
    """
    # Validate the method parameter
    if method not in {'TFIDF', 'LDA', 'BERT', 'LDA_BERT'}:
        raise ValueError("Invalid method! Choose from 'TFIDF', 'LDA', 'BERT', 'LDA_BERT'.")

    # Validate num_topics is an integer
    if not isinstance(num_topics, int):
        raise ValueError("num_topics must be an integer.")

    # Validate gamma is a number (int or float)
    if not isinstance(gamma, (int, float)):
        raise ValueError("gamma must be an integer or float.")

    # Regular expression pattern to match the file name based on the method, num_topics, and gamma
    pattern = rf'{method}_{data_file_name}_n_topics_{num_topics}_gamma_{gamma}_\d{{4}}_\d{{2}}_\d{{2}}_\d{{2}}_\d{{2}}_\d{{2}}\.file'

    directory_to_search = f'{model_files_path}/{method}'

    # Get all files in the directory matching the pattern
    model_files = [f for f in os.listdir(directory_to_search) if re.match(pattern, f)]

    if model_files:
        # Sort the files by the timestamp embedded in the filename
        latest_model_file = max(model_files, key=lambda x: re.findall(r'\d+', x))

        # Construct the full path to the latest model file
        model_path = os.path.join(directory_to_search, latest_model_file)

        # Load the TopicModel object from the file
        with open(model_path, "rb") as f:
            loaded_model = pickle.load(f)

        if isinstance(loaded_model, TopicModel):
            print(f"Successfully loaded the latest TopicModel with method '{method}', {num_topics} topics, and gamma '{gamma}': {latest_model_file}")
            return loaded_model
        else:
            print("The loaded file is not an instance of TopicModel.")
            return None
    else:
        print(f"No model files found for method '{method}' with {num_topics} topics and gamma '{gamma}'.")
        return None
        

https://chatgpt.com/share/672dbb3a-cf0c-800c-b1d5-0ff2d1a1c5fd

AUTOENCODER

In [ ]:
class SelfAttention(nn.Module):
    def __init__(self, dim):
        super(SelfAttention, self).__init__()
        self.attention = nn.MultiheadAttention(dim, num_heads=4, batch_first=True)

    def forward(self, x):
        attn_output, _ = self.attention(x, x, x)
        return attn_output + x  # Residual connection

class GatedLinearUnit(nn.Module):
    def __init__(self):
        super(GatedLinearUnit, self).__init__()

    def forward(self, x):
        return x * torch.sigmoid(x)

class Autoencoder(nn.Module):
    def __init__(self, lda_dim, embedding_dim, latent_dim=64):
        super(Autoencoder, self).__init__()
        
        # Separate encoders for LDA and Embeddings
        self.lda_encoder = nn.Sequential(
            nn.Linear(lda_dim, 128),
            nn.GELU(),
            SelfAttention(128),
            nn.Linear(128, latent_dim),
            nn.LayerNorm(latent_dim)
        )
        
        self.embedding_encoder = nn.Sequential(
            nn.Linear(embedding_dim, 256),
            nn.GELU(),
            nn.Conv1d(1, 1, kernel_size=3, padding=1),  # Convolution to capture local dependencies
            GatedLinearUnit(),
            SelfAttention(256),
            nn.Linear(256, latent_dim),
            nn.LayerNorm(latent_dim)
        )
        
        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(2 * latent_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, lda_dim + embedding_dim)  # Output for both LDA and embedding dimensions
        )

        # Variational layers
        self.fc_mu = nn.Linear(latent_dim, latent_dim)
        self.fc_logvar = nn.Linear(latent_dim, latent_dim)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, lda_input, embedding_input):
        # Encode LDA and embeddings separately
        lda_z = self.lda_encoder(lda_input)
        embedding_z = self.embedding_encoder(embedding_input.unsqueeze(1)).squeeze(1)
        
        # Reparameterization for structured latent space
        mu, logvar = self.fc_mu(lda_z), self.fc_logvar(lda_z)
        z = self.reparameterize(mu, logvar)
        
        # Concatenate encoded features and pass to decoder
        concatenated_z = torch.cat([z, embedding_z], dim=1)
        reconstructed = self.decoder(concatenated_z)
        
        return reconstructed, z, mu, logvar

class AutoencoderHandler():
    def __init__(self, lda_data, embeddings, scaled = False, latent_dim = 64, num_epochs = 24, batch_size = 64, checkpoint_dir="checkpoints", log_interval=100, data_file_name = data_file_name):
        self.lda_data = lda_data
        self.embeddings = embeddings
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.scaled = scaled
        self.latent_dim = latent_dim
        self.batch_size = batch_size
        self.num_epochs = num_epochs
        self.checkpoint_dir = checkpoint_dir
        self.log_interval = log_interval
        self.data_file_name = data_file_name
        self.get_scaled_data()
        self.lda_data = torch.tensor(self.lda_data, dtype=torch.float32).to(device)
        self.embeddings = torch.tensor(self.embeddings, dtype=torch.float32).to(device)
        self.lda_data = self.lda_data.to(device)
        self.embeddings = self.embeddings.to(device)
        self.dataset = TensorDataset(torch.tensor(self.scaled_lda_data).float(), torch.tensor(self.scaled_embeddings).float())
        self.dataloader = DataLoader(self.dataset, batch_size=self.batch_size, shuffle=True)
        self.model = Autoencoder(lda_dim=self.scaled_lda_data.shape[1], embedding_dim=self.scaled_embeddings.shape[1], latent_dim=self.latent_dim)
        self.model = self.model.to(device)
        self.optimizer = optim.AdamW(self.model.parameters(), lr=1e-4)

    def get_scaled_data(self):
        if self.scaled:
            self.scaled_lda_data = self.lda_data
            self.scaled_embeddings = self.embeddings
        else:
            self.lda_scaler = StandardScaler()
            self.embedding_scaler = StandardScaler()
            self.scaled_lda_data = self.lda_scaler.fit_transform(self.lda_data)
            self.scaled_embeddings = self.embedding_scaler.fit_transform(self.embeddings)
            save_vectors('scaled_lda', self.scaled_lda_data, self.data_file_name)
            save_vectors('scaled_embeddings', self.scaled_embeddings, self.data_file_name)

    # Define the contrastive loss with InfoNCE
    def contrastive_loss(self, z_anchor, z_positive, z_negative, margin=1.0):
        pos_dist = nn.functional.cosine_similarity(z_anchor, z_positive)
        neg_dist = nn.functional.cosine_similarity(z_anchor, z_negative)
        return torch.mean(torch.relu(margin + neg_dist - pos_dist))

    # Combined Loss Function
    def combined_loss(self, reconstructed, original, z_anchor, z_positive, z_negative, mu, logvar, epoch, max_epochs, beta=0.01, contrastive_weight=0.1):
        reconstruction_loss = nn.MSELoss()(reconstructed, original)
        kl_divergence = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
        contrast_loss = self.contrastive_loss(z_anchor, z_positive, z_negative)
        dynamic_contrastive_weight = contrastive_weight * (epoch / max_epochs)
        
        return reconstruction_loss + beta * kl_divergence + dynamic_contrastive_weight * contrast_loss

    # Training function with progress logging
    def train(self):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.train()
        scaler = GradScaler()  # For mixed precision
        best_loss = float("inf")
        latent_reps = []

        for epoch in range(self.num_epochs):
            epoch_loss = 0
            for batch_idx, (lda_input, song_embedding) in enumerate(self.dataloader):
                lda_input, song_embedding = lda_input.to(device), song_embedding.to(device)
                self.optimizer.zero_grad()

                # Forward pass with autocast for mixed precision
                with autocast():
                    reconstructed, z_anchor, mu, logvar = self.model(lda_input, song_embedding)
                    latent_reps.append(z_anchor.detach().cpu().numpy())
                    
                    # Generate positive and negative pairs
                    positive_idx = (batch_idx + 1) % len(self.dataloader)
                    negative_idx = (batch_idx + 2) % len(self.dataloader)
                    lda_pos, emb_pos = self.dataloader.dataset[positive_idx]
                    lda_neg, emb_neg = self.dataloader.dataset[negative_idx]
                    lda_pos, emb_pos = lda_pos.to(device), emb_pos.to(device)
                    lda_neg, emb_neg = lda_neg.to(device), emb_neg.to(device)
                    
                    _, z_positive, _, _ = self.model(lda_pos.unsqueeze(0), emb_pos.unsqueeze(0))
                    _, z_negative, _, _ = self.model(lda_neg.unsqueeze(0), emb_neg.unsqueeze(0))
                    
                    # Calculate combined loss
                    loss = self.combined_loss(reconstructed, torch.cat((lda_input, song_embedding), dim=1), 
                                        z_anchor, z_positive, z_negative, mu, logvar, epoch, self.num_epochs, beta=0.01)

                # Backward pass with gradient scaling
                scaler.scale(loss).backward()
                scaler.step(self.optimizer)
                scaler.update()
                
                epoch_loss += loss.item()

                if batch_idx % self.log_interval == 0:
                    print(f"Epoch [{epoch+1}/{self.num_epochs}], Batch [{batch_idx}], Loss: {loss.item():.4f}")

            avg_epoch_loss = epoch_loss / len(self.dataloader)
            print(f"Epoch [{epoch+1}/{self.num_epochs}], Avg Loss: {avg_epoch_loss:.4f}")

            # Save model if it achieves the best loss
            if avg_epoch_loss < best_loss:
                best_loss = avg_epoch_loss
                torch.save(self.model.state_dict(), f"{self.checkpoint_dir}/best_model.pth")
                print(f"Checkpoint saved with Avg Loss: {avg_epoch_loss:.4f}")
        latent_representations = np.concatenate(latent_reps)
        save_vectors('autoencoder_latent_representations', latent_representations, self.data_file_name)
        



MODEL

In [ ]:
# define model object

class TopicModel:
    def __init__(self, 
            k=25,
            alpha=0.01, 
            beta=0.5,
            data_file_name=None,
            token_lists=None,
            sentences=None,
            model_name=None,
            use_n_grams=True,
            tfidf_input_path=tfidf_input_path,
            lda_vec_input_path=lda_input_path,
            bert_input_path=song_embeddings_input_path
        ):
        """
        Initialize the TopicModel class.
        """
        self.k = k
        self.alpha = alpha
        self.beta = beta
        self.print_message(f'Initialized TopicModel with k={k}, alpha={alpha}, beta={beta}')
        self.dictionary = None
        self.corpus = None
        self.cluster_model = None
        self.ldamodel = None
        self.vec = {}
        self.use_n_grams = use_n_grams
        self.data_file_name = data_file_name  # Added data_file_name initialization
        self.id = f'{data_file_name}_n_topics_{k}_{datetime.now().strftime("%Y_%m_%d_%H_%M_%S")}'
        self.model_name = model_name
        self.model_name_print = f'{model_name} Topic model' if model_name else 'Default Topic model'
        self.token_lists = token_lists
        self.sentences = sentences
        self.tfidf_corpus_in_path = f'{tfidf_input_path}/tfidf_corpus.file'
        self.tfidf_dict_in_path = f'{tfidf_input_path}/tfidf_dict.file'
        self.lda_model_in_path = f'{lda_input_path}/LDA_model_{data_file_name}.model'
        self.lda_vec_in_path = f'{lda_vec_input_path}/LDA_{data_file_name}.file'
        if os.path.exists(self.lda_model_in_path):
            self.print_message(f'LDA model found at {self.lda_model_in_path}')
        else:
            self.print_message(f'LDA model not found at {self.lda_model_in_path}')
        bert_name = f'{model_name}_Song_embeddings_preprocessed_sentences' if model_name else 'Song_embeddings_preprocessed_sentences'
        self.bert_vec_in_path = f'{bert_input_path}/{bert_name}_{data_file_name}.file'
        self.print_message("initialized")

    def print_message(self, message):
        print(f"{self.model_name_print}: {message}")

    def load_tfidf(self):
        """
        Load TF-IDF vectors.
        """
        self.print_message('Loading TF-IDF vectors ...')
        if os.path.exists(self.tfidf_corpus_in_path) and os.path.exists(self.tfidf_dict_in_path):
            with open(self.tfidf_corpus_in_path, 'rb') as f:
                self.corpus = pickle.load(f)
            self.dictionary = corpora.Dictionary.load(self.tfidf_dict_in_path.replace('pkl', 'dict'))
            self.print_message('Loaded TF-IDF vectors')
            return True
        else:
            self.print_message('TF-IDF vectors not found')
            return False
        
    def save_tfidf(self):
        """
        Save TF-IDF vectors.
        """
        tfidf_name = f'{self.model_name}_tfidf' if self.model_name else 'tfidf'
        save_vectors(tfidf_name, self.corpus, 'corpus')
        save_vectors(tfidf_name, self.dictionary, 'dict')
        self.tfidf_model.save(f'{vector_files_path}/{tfidf_name}_model.model')
        self.dictionary.save(f'{vector_files_path}/{tfidf_name}_dict.dict')
        self.print_message('Saved TF-IDF vectors')
    
    def vectorize_tfidf(self, no_below=5, no_above=0.5):
        """
        Vectorize TF-IDF vectors.
        """
        if not self.load_tfidf():
            if self.use_n_grams:
                self.print_message('Creating n-grams ...')
                bigram_phrases = gensim.models.Phrases(self.token_lists, min_count=5, threshold=100)
                trigram_phrases = gensim.models.Phrases(bigram_phrases[self.token_lists], threshold=100)
                
                bigram = gensim.models.phrases.Phraser(bigram_phrases)
                trigram = gensim.models.phrases.Phraser(trigram_phrases)

                bigram_name = f'{self.model_name}_bigram' if self.model_name else 'bigram'
                trigram_name = f'{self.model_name}_trigram' if self.model_name else 'trigram'
                n_grams_name = f'{self.model_name}_n_grams' if self.model_name else 'n_grams'

                bigram.save(f'{vector_files_path}/{bigram_name}.phr')
                trigram.save(f'{vector_files_path}/{trigram_name}.phr')

                def make_bigrams(texts):
                    return([bigram[doc] for doc in texts])

                def make_trigrams(texts):
                    return ([trigram[bigram[doc]] for doc in texts])

                data_bigrams = make_bigrams(self.token_lists)
                data_bigrams_trigrams = make_trigrams(data_bigrams)
                self.token_lists = data_bigrams_trigrams

                save_vectors(n_grams_name, self.token_lists, 'n_grams_token_lists.pkl')

                self.print_message('Created n-grams')
                
            self.print_message('Creating TF-IDF vectors ...')
            dictionary = corpora.Dictionary(self.token_lists)
            dictionary.filter_extremes(no_below=no_below, no_above=no_above)  
            corpus = [dictionary.doc2bow(doc) for doc in self.token_lists]
            self.tfidf_model = TfidfModel(corpus)
            tfidf_corpus = [self.tfidf_model[doc] for doc in corpus]
            self.corpus = tfidf_corpus
            self.dictionary = dictionary
            self.print_message('Created TF-IDF vectors')
            self.save_tfidf()
    
    def load_lda(self, load_id2word = False):
        """
        Load LDA model and vectors.
        """
        if os.path.exists(self.lda_model_in_path) and os.path.exists(self.lda_vec_in_path):
            # self.ldamodel = gensim.models.LdaModel.load(self.lda_model_in_path)
            lda_model_file = self.lda_model_in_path.replace('.model', '.file')
            with open(lda_model_file, 'rb') as f:
                self.ldamodel = pickle.load(f)
            with open(self.lda_vec_in_path, 'rb') as f:
                self.vec['LDA'] = pickle.load(f)
            if load_id2word:
                self.dictionary = corpora.Dictionary.load(f'{self.lda_model_in_path}.id2word')
                self.corpus = [self.dictionary.doc2bow(doc) for doc in self.token_lists]
            self.print_message('Loaded LDA model and vectors')
            return 1
        elif os.path.exists(self.lda_model_in_path):
            self.ldamodel = gensim.models.LdaModel.load(self.lda_model_in_path)
            self.print_message('LDA model found, but vectors not found')
            return 2
        else:
            self.print_message('LDA model and vectors not found')
            return 3
        
    def get_lda_vectors_from_model(self):
        self.print_message('Creating LDA vectors')
        n_doc = len(self.corpus)
        vec_lda = np.zeros((n_doc, self.k))
        for i in range(n_doc):
            # Get the distribution for the i-th document in corpus
            for topic, prob in self.ldamodel.get_document_topics(self.corpus[i]):
                vec_lda[i, topic] = prob
        self.vec['LDA'] = vec_lda
        self.print_message('Created LDA vectors')
        self.save_lda()
        self.evaluate_lda_model()

    def get_lda_vectors(self):
        """
        Get LDA vector representation based on the existing corpus.
        """
        status = self.load_lda()
        if status == 1:
            return self.ldamodel, self.vec['LDA']
        elif status == 2:
            self.vectorize_tfidf()
            self.get_lda_vectors_from_model()
            return self.ldamodel, self.vec['LDA']
        else:
            self.vectorize_tfidf()
            if not self.ldamodel:
                self.print_message('Creating LDA model')
                # self.ldamodel = gensim.models.LdaModel(corpus=self.corpus, num_topics=self.k, id2word=self.dictionary, passes=20)
                self.ldamodel = gensim.models.LdaMulticore(corpus=self.corpus, num_topics=self.k, id2word=self.dictionary, workers=2, passes=20, random_state=42, alpha=self.alpha, eta=self.beta)
                self.print_message('Created LDA model')
            self.get_lda_vectors_from_model()
            return self.ldamodel, self.vec['LDA']
    
    def save_lda(self):
        """
        Save LDA model and vectors.
        """
        lda_name = f'{self.model_name}_LDA' if self.model_name else 'LDA'
        self.ldamodel.save(f'{vector_files_path}/{lda_name}_model_{self.data_file_name}.model')
        save_vectors(f'{lda_name}_model', self.ldamodel, self.data_file_name)
        save_vectors(lda_name, self.vec['LDA'], self.data_file_name)
        self.print_message('Saved LDA model and vectors')

    def evaluate_lda_model(self, coherence_type='c_v'):
        """
        Evaluate the LDA model.
        """
        self.print_message('Evaluating LDA model ...')
        # self.load_tfidf()
        # self.load_lda(load_id2word = False)
        self.evaluate_perplexity()
        self.evaluate_coherence(coherence_type)
        # self.visualize_lda()

    def evaluate_perplexity(self):
        """
        Evaluate the perplexity of the LDA model.
        """
        if not self.ldamodel or not self.corpus:
            self.print_message("LDA model or corpus is not available for perplexity evaluation.")
            return None
        
        perplexity = self.ldamodel.log_perplexity(self.corpus)
        self.print_message(f"Perplexity: {perplexity}")
        return perplexity

    def evaluate_coherence(self, coherence_type='c_v'):
        """
        Evaluate the coherence score of the LDA model.
        :param coherence_type: The type of coherence measure to use (default is 'c_v').
        """
        if not self.ldamodel or not self.token_lists or not self.dictionary:
            self.print_message("LDA model, token lists, or dictionary is not available for coherence evaluation.")
            return None
        
        coherence_model = CoherenceModel(model=self.ldamodel, 
                                         texts=self.token_lists, 
                                         dictionary=self.dictionary, 
                                         coherence=coherence_type)
        coherence_score = coherence_model.get_coherence()
        self.print_message(f"Coherence Score ({coherence_type}): {coherence_score}")
        return coherence_score

    def visualize_lda(self):
        """
        Visualizes the LDA model using pyLDAvis.
        """
        if not self.ldamodel or not self.dictionary or not self.corpus:
            self.print_message("LDA model, dictionary, or corpus is not available for visualization.")
            return
        
        # Prepare the pyLDAvis visualization
        vis = pyLDAvis.gensim_models.prepare(self.ldamodel, self.corpus, self.dictionary)
        
        # Display the visualization (works well in Jupyter notebooks)
        # pyLDAvis.display(vis)
        
        # Save the visualization to an HTML file
        lda_name = f'{self.model_name}_LDA' if self.model_name else 'LDA'
        vis_file = f'{vector_files_path}/{lda_name}_{self.data_file_name}_visualization.html'
        vis.save_html(vis_file)
        self.print_message(f"Saved pyLDAvis visualization to {vis_file}")

    # Method for hyperparameter tuning of LDA model
    def tune_lda_hyperparameters(self, k_values=[5, 10, 15], alpha_values=[0.01, 0.1, 0.5], beta_values=[0.01, 0.1, 0.5], passes=20):
        """
        Perform hyperparameter tuning for LDA model.
        
        :param k_values: List of number of topics to try.
        :param alpha_values: List of alpha values to try.
        :param beta_values: List of beta values to try.
        :param passes: Number of passes for the LDA model.
        """
        self.vectorize_tfidf()
        best_coherence = -np.inf
        best_params = {'k': None, 'alpha': None, 'beta': None}
        best_lda_model = None

        configurations = [
            (20, 0.01, 0.01),
            (20, 0.01, 0.5),
            (20, 0.1, 0.5),
            (20, 0.5, 0.5),
            (25, 0.01, 0.01),
            (25, 0.01, 0.5),
            (25, 0.1, 0.5),
            (25, 0.5, 0.5),
            (15, 0.01, 0.5)
        ]
        
        for k, alpha, beta in configurations:
            self.print_message(f"Tuning with k={k}, alpha={alpha}, beta={beta}")
            
            # Train LDA model with current hyperparameters
            self.ldamodel = gensim.models.LdaMulticore(
                corpus=self.corpus,
                num_topics=k,
                id2word=self.dictionary,
                alpha=alpha,
                eta=beta,
                passes=passes,
                workers=2,
                random_state=42
            )

            # Evaluate the coherence score
            coherence_score = self.evaluate_coherence(coherence_type='c_v')

            # Update the best model if needed
            if coherence_score > best_coherence:
                best_coherence = coherence_score
                best_params = {'k': k, 'alpha': alpha, 'beta': beta}
                best_lda_model = self.ldamodel

            self.print_message(f"Best Coherence Score: {best_coherence}")
            self.print_message(f"Best Hyperparameters: {best_params}")

        # Loop through all combinations of hyperparameters
        # for k in k_values:
        #     for alpha in alpha_values:
        #         for beta in beta_values:
        #             self.print_message(f"Tuning with k={k}, alpha={alpha}, beta={beta}")
                    
        #             # Train LDA model with current hyperparameters
        #             self.ldamodel = gensim.models.LdaMulticore(
        #                 corpus=self.corpus,
        #                 num_topics=k,
        #                 id2word=self.dictionary,
        #                 alpha=alpha,
        #                 eta=beta,
        #                 passes=passes,
        #                 workers=2,
        #                 random_state=42
        #             )

        #             # Evaluate the coherence score
        #             coherence_score = self.evaluate_coherence(coherence_type='c_v')

        #             # Update the best model if needed
        #             if coherence_score > best_coherence:
        #                 best_coherence = coherence_score
        #                 best_params = {'k': k, 'alpha': alpha, 'beta': beta}
        #                 best_lda_model = self.ldamodel

        #             self.print_message(f"Best Coherence Score: {best_coherence}")
        #             self.print_message(f"Best Hyperparameters: {best_params}")
        
        # Save the best LDA model
        if best_lda_model:
            self.ldamodel = best_lda_model
            self.get_lda_vectors_from_model()
            self.save_lda()
            self.print_message(f"Saved best LDA model with {best_params}")
        return best_params, best_coherence
        
    def load_bert(self):
        """
        Load BERT vectors.
        """
        if os.path.exists(self.bert_vec_in_path):
            with open(self.bert_vec_in_path, 'rb') as f:
                self.vec['BERT'] = pickle.load(f)
            self.print_message('Loaded Song embeddings')
            return True
        else:
            self.print_message('Song embeddings not found')
            return False
    
    def get_bert_vectors(self):
        """
        Get BERT vector representations.
        """
        if self.load_bert():
            return self.vec['BERT']
        else:
            self.print_message('Creating Song embeddings ...')
            # model = SentenceTransformer('bert-base-nli-max-tokens')
            model = SentenceTransformer('all-mpnet-base-v2')
            vec = np.array(model.encode(self.token_lists, show_progress_bar=True))
            self.vec['BERT'] = vec
            self.print_message('Created Song embeddings')
            self.save_bert()
            return vec
    
    def save_bert(self):
        """
        Save BERT vectors.
        """
        bert_name = f'{self.model_name}_Song_embeddings_preprocessed_sentences' if self.model_name else 'Song_embeddings_preprocessed_sentences'
        save_vectors(bert_name, self.vec['BERT'], self.data_file_name)
        self.print_message('Saved Song embeddings')

    def get_lda_bert_vectors(self):
        """
        Get LDA and BERT vector representations.
        """
        self.get_lda_vectors()
        self.get_bert_vectors()
        return self.vec['LDA'], self.vec['BERT']


In [ ]:
class AdvancedSongRecommender:
    def __init__(self, latent_representations: np.ndarray, reduced_dim_latent_representations: np.ndarray, song_lyrics: List[str], top_k: int = 50, cache_dir_in=song_recommender_cache_input_path, cache_dir_out=song_recommender_cache_out_path):
        """
        Initialize the recommender with latent representations and lyrics data.

        Parameters:
        - latent_representations: Latent representations of songs.
        - song_lyrics: List of lyrics corresponding to each song.
        - top_k: Number of top recommendations to return.
        - cache_dir_in: Directory where cache files will be read from.
        - cache_dir_out: Directory where cache files will be saved.
        """
        self.latent_representations = latent_representations
        self.reduced_dim_latent_representations = reduced_dim_latent_representations
        self.song_lyrics = song_lyrics
        self.top_k = top_k
        self.index_file_in = f"{cache_dir_in}/faiss_index.idx"
        self.cluster_file_in = f"{cache_dir_in}/cluster_labels.npy"
        self.index_file_out = f"{cache_dir_out}/faiss_index.idx"
        self.cluster_file_out = f"{cache_dir_out}/cluster_labels.npy"
        self.theme_cache_file_in = f"{cache_dir_in}/cluster_themes.pkl"
        self.theme_cache_file_out = f"{cache_dir_out}/cluster_themes.pkl"

        print("Building FAISS index for global candidate retrieval")
        # Step 1: FAISS Index for Global Candidate Retrieval
        self.faiss_index = self._build_faiss_index()
        
        print("Building HDBSCAN clusters")
        # Step 2: HDBSCAN Global Clustering (precomputed for theme clusters)
        self.cluster_labels = self._build_hdbscan_clusters()
        
        print("Storing songs in each cluster for fast retrieval")
        # Store songs in each cluster for fast retrieval
        self.cluster_members = self._create_cluster_members()

        # print("Building Gensim LDA model for theme extraction")
        # # Step 3: Build Gensim LDA model for theme extraction
        # self.lda_model, self.dictionary = self._train_gensim_lda_model()

        # print("Precomputing and caching cluster themes")
        # # Step 4: Precompute and cache cluster themes during initialization
        # self.theme_cache = self._precompute_and_cache_themes()

    def _build_faiss_index(self):
        """Build or load a FAISS index with optimized parameters for a large dataset."""
        if not self._load_faiss_index():  # Load existing index if available
            d = self.latent_representations.shape[1]
            nlist = 4096  # Number of Voronoi cells
            m = 16  # Number of sub-quantizers for PQ
            index = faiss.IndexIVFPQ(faiss.IndexFlatIP(d), d, nlist, m, 8)  # 8 bits per sub-vector
            index.train(self.latent_representations)
            index.add(self.latent_representations)
            index.nprobe = 20  # Number of clusters to search during querying
            faiss.write_index(index, self.index_file_out)  # Save for future use
            print("FAISS index built and saved.")
        return index

    def _load_faiss_index(self):
        """Load the FAISS index if it exists."""
        try:
            self.faiss_index = faiss.read_index(self.index_file_in)
            print("FAISS index loaded from disk.")
            return True
        except:
            print("No pre-existing FAISS index found; building a new one.")
            return False

    def _build_hdbscan_clusters(self):
        """Build or load HDBSCAN clustering model."""
        if not self._load_clusters():
            hdbscan_clusterer = hdbscan.HDBSCAN(min_cluster_size=1000, min_samples=100, metric='euclidean')
            cluster_labels = hdbscan_clusterer.fit_predict(self.reduced_dim_latent_representations)
            np.save(self.cluster_file_out, cluster_labels)  # Save cluster labels for future use
            print("HDBSCAN clustering done and saved.")
        return cluster_labels

    def _load_clusters(self):
        """Load precomputed cluster labels if available."""
        try:
            self.cluster_labels = np.load(self.cluster_file_in)
            print("Cluster labels loaded from disk.")
            return True
        except:
            print("No pre-existing cluster labels found; building clusters.")
            return False

    def _create_cluster_members(self) -> Dict[int, List[int]]:
        """Organize songs into clusters for fast cluster-based retrieval."""
        cluster_dict = {}
        for idx, label in enumerate(self.cluster_labels):
            if label != -1:  # Ignore noise points
                cluster_dict.setdefault(label, []).append(idx)
        return cluster_dict

    def _train_gensim_lda_model(self) -> tuple:
        """Train a Gensim LDA model to extract topics/themes from song lyrics."""
        # Tokenize the lyrics (simple whitespace tokenizer)
        texts = [song.split() for song in self.song_lyrics]
        
        # Create a dictionary and corpus for Gensim LDA model
        dictionary = Dictionary(texts)
        corpus = [dictionary.doc2bow(text) for text in texts]

        # Train the LDA model with 10 topics
        lda_model = LdaModel(corpus, num_topics=10, id2word=dictionary, passes=15)
        print("Gensim LDA model trained for theme extraction.")
        return lda_model, dictionary

    def _precompute_and_cache_themes(self) -> Dict[int, str]:
        """Precompute and cache the themes for all clusters during initialization."""
        theme_cache = self._load_theme_cache()
        if theme_cache:
            return theme_cache
        else:
            theme_cache = {}
            for cluster_id in set(self.cluster_labels):
                if cluster_id != -1:  # Exclude noise points
                    theme_cache[cluster_id] = self._get_cluster_theme(cluster_id)
            # Save the cached themes to disk for later use
            self._save_theme_cache(theme_cache)
            print("Cluster themes precomputed and cached.")
            return theme_cache
    
    def _load_theme_cache(self):
        """Load precomputed cluster themes if available."""
        if os.path.exists(self.theme_cache_file_in):
            with open(self.theme_cache_file_in, 'rb') as f:
                theme_cache = pickle.load(f)
            print("Cluster themes loaded from disk.")
            return theme_cache
        else:
            print("No pre-existing cluster themes found; building clusters.")
            return None

    def _get_cluster_theme(self, cluster_id: int) -> str:
        """Generate a theme summary for a given cluster using Gensim LDA model."""
        cluster_songs = self.cluster_members.get(cluster_id, [])
        lyrics_for_cluster = [self.song_lyrics[song_id] for song_id in cluster_songs]

        # Tokenize the songs' lyrics for LDA analysis
        texts_for_cluster = [song.split() for song in lyrics_for_cluster]
        corpus_for_cluster = [self.dictionary.doc2bow(text) for text in texts_for_cluster]

        # Get the topics distribution for the cluster's songs
        topic_distribution = self.lda_model[corpus_for_cluster]

        # Collect the most frequent terms for the top topics
        top_topics = [max(doc, key=lambda x: x[1])[0] for doc in topic_distribution]
        topic_keywords = []
        for topic_idx in top_topics:
            keywords = [word for word, _ in self.lda_model.show_topic(topic_idx, topn=5)]
            topic_keywords.append(", ".join(keywords))

        # Summarize the theme of the cluster
        theme_summary = " | ".join(topic_keywords)
        return theme_summary

    def _save_theme_cache(self, theme_cache: Dict[int, str]):
        """Save the theme cache to disk."""
        with open(self.theme_cache_file_out, 'wb') as f:
            pickle.dump(theme_cache, f)
        print("Theme cache saved.")

    def recommend(self, song_id: int, novelty: float = 0.5) -> List[int]:
        """
        Generate recommendations with adjustable novelty using FAISS and cluster-based filtering.

        Parameters:
        - song_id: ID of the song to base recommendations on.
        - novelty: Control the diversity (0 = similar songs, 1 = diverse songs).

        Returns:
        - List of recommended song IDs with theme explanation.
        """
        print("Starting recommendations for song_id %d", song_id)

        # Step 1: Retrieve top-k global candidates from FAISS
        _, top_candidates = self.faiss_index.search(self.latent_representations[song_id:song_id+1], self.top_k)
        # candidate_reps = self.latent_representations[top_candidates.flatten()]
        
        # Step 2: Cluster-Based Filtering
        target_cluster = self.cluster_labels[song_id]
        main_cluster_songs = self.cluster_members.get(target_cluster, [])
        
        # Adjust clusters based on novelty
        if novelty < 0.5:
            # Low novelty: Prioritize songs within main cluster
            filtered_candidates = [song for song in top_candidates.flatten() if song in main_cluster_songs]
        else:
            # High novelty: Include songs from neighboring clusters
            nearby_clusters = self._get_nearby_clusters(target_cluster, novelty)
            filtered_candidates = [song for song in top_candidates.flatten() if song in nearby_clusters]

        # Step 3: Intra-Cluster Ranking (Cosine similarity)
        scores = cosine_similarity(
            self.reduced_dim_latent_representations[song_id:song_id+1],
            self.reduced_dim_latent_representations[filtered_candidates]
        ).flatten()
        ranked_candidates = [filtered_candidates[i] for i in np.argsort(scores)]

        # Step 4: Get Themes and Explanation
        theme_explanations = []
        for song in ranked_candidates:
            theme = self._get_cluster_theme(self.cluster_labels[song])
            theme_explanations.append(theme)

        return ranked_candidates[:self.top_k], theme_explanations

    def _get_nearby_clusters(self, target_cluster: int, novelty_factor: float = 0.5) -> List[int]:
        """Get nearby clusters based on centroids' distances (simulate diverse recommendations)."""
        cluster_distances = {
            label: np.linalg.norm(
                np.mean(self.reduced_dim_latent_representations[self.cluster_members[label]], axis=0) -
                np.mean(self.reduced_dim_latent_representations[self.cluster_members[target_cluster]], axis=0)
            )
            for label in self.cluster_members
        }
        sorted_clusters = sorted(cluster_distances.items(), key=lambda x: x[1])
        
        # Adjust number of clusters included based on novelty
        max_clusters = int(len(sorted_clusters) * novelty_factor)
        return [label for label, _ in sorted_clusters[:max_clusters]]

PREPROCESS DATA

In [ ]:
# if not load_existing_preprocessed_data:
#     print('Getting lyrics')
#     data = documents #pd.read_csv(working_files_path + '/train.csv')
#     data = data.fillna('')  # only the comments has NaN's
#     rws = data.lyrics
#     print(type(rws))

In [ ]:
# sentences, token_lists, idx_in = load_progress()

In [ ]:
# if load_existing_preprocessed_data:
#     sentences, token_lists = load_preprocessed_files(data_file_name)
# else:
#     sentences, token_lists, idx_in = preprocess(rws)
# print(type(sentences), type(token_lists))
# # print(np.shape(sentences), np.shape(token_lists))

In [ ]:
if load_preprocessed_checkpoints:
    print('Loading preprocessed train checkpoints')
    train_sentences, train_token_lists, train_idx_in, train_ngram_token_lists = load_progress()
    print('Loaded preprocessed train checkpoints')
    print(len(train_idx_in), len(train_sentences), len(train_token_lists))
else:
    print('Enable load_preprocessed_checkpoints')

In [ ]:
def reduce_dim_umap(latent_representations, n_components=30):
    # Initialize UMAP model
    umap_model = umap.UMAP(n_components=n_components, metric='euclidean')

    # Fit UMAP on the entire dataset
    final_embeddings = umap_model.fit_transform(latent_representations)

    # Save the final embeddings
    np.save("final_reduced_dim_latent_representations.npy", final_embeddings)
    
    return final_embeddings


In [ ]:
if load_existing_model:
    tm = load_latest_topic_model(num_topics = ntopic, gamma = gamma)
else:
    # Define the topic model object
    #tm = TopicModel(k = 10), method = TFIDF)
    # tm = TopicModel(k = ntopic, gamma = gamma, eps = db_scan_eps, min_samp = db_scan_min_samples, data_file_name = 'data_file_name')
    # Fit the topic model by chosen method
    # tm.vectorize_tfidf(sentences, max_df = 0.5)
    
    # tm = TopicModel(
    #     data_file_name = data_file_name,
    #     token_lists = train_ngram_token_lists,
    #     sentences = train_sentences
    # )
    # tm.get_lda_bert_vectors()
    # tm.evaluate_lda_model()
    # tm.get_lda_vectors()
    # tm.tune_lda_hyperparameters()
    

    print('Loading preprocessed val checkpoints')
    val_sentences, val_token_lists, val_idx_in, val_ngram_token_lists = load_progress(val_checkpoints_input_path)
    print('Loaded preprocessed val checkpoints')
    tm_val = TopicModel(
        data_file_name = data_file_name,
        token_lists = val_token_lists,
        sentences = val_sentences,
        model_name = 'val',
        tfidf_input_path = val_tfidf_input_path,
        lda_vec_input_path = val_lda_input_path,
        bert_input_path = val_song_embeddings_input_path,
    )
    tm_val.get_lda_vectors()
    
    # tm.vectorize_tfidf_2(token_lists)
    # tm.load_tfidf()
    # vec_lda = tm.get_lda_vectors()
    # save_vectors('LDA', vec_lda, tm.data_file_name)
    # print('LDA vectors saved')

    # vec_bert = tm.get_bert_vectors(sentences)
    # save_vectors('BERT_preprocessed_sentences', vec_bert, tm.data_file_name)
    # print('BERT vectors saved')
    # vec_lda = load_vectors_from_path(lda_input_path)
    # song_embeddings = load_vectors_from_path(song_embeddings_input_path)
    # autoencoder_handler = AutoencoderHandler(vec_lda, song_embeddings, scaled = False, latent_dim=64, num_epochs=24, batch_size=64, checkpoint_dir=autoencoder_checkpoint_dir, log_interval=100, data_file_name=data_file_name)
    # autoencoder_handler.train()

    # latent_representations = load_vectors_from_path(latent_representations_input_path)
    # reduced_dim_latent_representations = reduce_dim_umap(latent_representations)
    # lyrics = load_vectors_from_path(reordered_lyrics_input_path)
    # recommender = AdvancedSongRecommender(latent_representations, reduced_dim_latent_representations, lyrics)
    # recommendations = recommender.recommend(song_id=10, novelty=0.6)
    # logging.info("Recommended songs for song_id 10: %s", recommendations)

    # Assume unseen_song_rep is the latent representation of a new song
    # unseen_song_rep = np.random.rand(64)
    # recommendations, theme_explanations = recommender.recommend_for_unseen_song(unseen_song_rep, novelty=0.6)
    # logging.info("Recommended songs for unseen song: %s", recommendations)
    # logging.info("Theme explanations: %s", theme_explanations)
    


In [ ]:
print('done')